In [80]:
import pandas as pd

df = pd.read_csv("dataset.csv")

In [81]:
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'], format="%m/%d/%y")

In [82]:
df['year'] = df['ORDERDATE'].dt.year
df['month'] = df['ORDERDATE'].dt.month

In [83]:
df["UNITPRICE"] = df["UNITPRICE"] * 25_000

In [84]:
df.head()

,ROWID,ORDERID,ORDERDATE,ORDERPRIOTORITY,ORDERQUANTITY,SALES,DISCOUNT,SHIPMODE,PROFIT,UNITPRICE,...,CUSTOMERSEGMENT,PRODUCTCATEGORY,PRODUCTSUBCATEGORY,PRODUCTNAME,PRODUCTCONTAINER,PRODUCTBASEMARGIN,SHIPDATE,DATASET,year,month
0,1,3,2010-10-13,Low,6,261.5400,0.04,Regular Air,-213.25,973500.0,...,Small Business,Office Supplies,Storage & Organization,"Eldon Base for stackable storage shelf, platinum",Large Box,0.80,10/20/10,Superstore,2010,10
1,49,293,2012-10-01,High,49,10123.0200,0.07,Delivery Truck,457.81,5204000.0,...,Consumer,Office Supplies,Appliances,"1.7 Cubic Foot Compact ""Cube"" Office Refrigera...",Jumbo Drum,0.58,10/2/12,Superstore,2012,10
2,50,293,2012-10-01,High,27,244.5700,0.01,Regular Air,46.71,217250.0,...,Consumer,Office Supplies,Binders and Binder Accessories,"Cardinal Slant-DÂ Ring Binder, Heavy Gauge Vinyl",Small Box,0.39,10/3/12,Superstore,2012,10
3,80,483,2011-07-10,High,30,4965.7595,0.08,Regular Air,1198.97,4899750.0,...,Corporate,Technology,Telephones and Communication,R380,Small Box,0.58,7/12/11,Superstore,2011,7
4,85,515,2010-08-28,Not Specified,19,394.2700,0.08,Regular Air,30.94,544500.0,...,Consumer,Office Supplies,Appliances,Holmes HEPA Air Purifier,Medium Box,0.50,8/30/10,Superstore,2010,8


In [85]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

df["variant_id"] = encoder.fit_transform(df['PRODUCTNAME'])

In [86]:
import numpy as np

monthly = df.groupby(['variant_id', 'year', 'month']).agg({
    'ORDERQUANTITY': 'sum',
    'UNITPRICE': 'mean',
    "SALES": "sum"
}).reset_index()

monthly['quarter'] = np.ceil(monthly['month'] / 3).astype(int)
monthly['sin_quarter'] = np.sin(2 * np.pi * monthly['quarter'] / 4)
monthly['cos_quarter'] = np.cos(2 * np.pi * monthly['quarter'] / 4)

In [87]:
monthly.head()

,variant_id,year,month,ORDERQUANTITY,UNITPRICE,SALES,quarter,sin_quarter,cos_quarter
0,0,2009,1,42,92750.0,151.35,1,1.000000e+00,6.123234e-17
1,0,2010,5,39,92750.0,147.46,2,1.224647e-16,-1.000000e+00
2,0,2010,12,14,92750.0,51.56,4,-2.449294e-16,1.000000e+00
3,0,2011,6,13,92750.0,49.08,2,1.224647e-16,-1.000000e+00
4,0,2011,12,16,92750.0,60.02,4,-2.449294e-16,1.000000e+00


In [88]:
monthly['view'] = 0
monthly['add_to_cart'] = 0

In [89]:
monthly['best_seller_of_month'] = monthly.groupby(['year','month'])['ORDERQUANTITY']\
                                         .transform(lambda x: x == x.max()).astype(int)


In [90]:
monthly = monthly.sort_values(by=['variant_id', 'year', 'month'])

monthly['orderquantity_next_month'] = monthly.groupby('variant_id')['ORDERQUANTITY'].shift(-1)
monthly['view_next_month'] = monthly.groupby('variant_id')['view'].shift(-1)
monthly['add_to_cart_next_month'] = monthly.groupby('variant_id')['add_to_cart'].shift(-1)
monthly['best_seller_next_month'] = monthly.groupby('variant_id')['best_seller_of_month'].shift(-1)


In [91]:
monthly = monthly.dropna()

In [92]:
quarter = monthly.groupby(['variant_id', 'quarter', 'year']).agg({
    'ORDERQUANTITY': 'sum',
    'UNITPRICE': 'mean',
    "SALES": "sum",
    "view": "sum",
    "add_to_cart": "sum",
    
}).reset_index()
quarter = quarter.sort_values(by=['variant_id', 'year', 'quarter'], ascending=True)
quarter['orderquantity_next_quarter'] = quarter.groupby('variant_id')['ORDERQUANTITY'].shift(-1)
quarter['view_next_quarter'] = quarter.groupby('variant_id')['view'].shift(-1)
quarter['add_to_cart_next_quarter'] = quarter.groupby('variant_id')['add_to_cart'].shift(-1)

quarter['best_seller_of_quarter'] = quarter.groupby(['year','quarter'])['ORDERQUANTITY']\
                                         .transform(lambda x: x == x.max()).astype(int)
quarter['best_seller_next_quarter'] = quarter.groupby('variant_id')['best_seller_of_quarter'].shift(-1)

quarter = quarter.dropna()
quarter.head(n=30)

,variant_id,quarter,year,ORDERQUANTITY,UNITPRICE,SALES,view,add_to_cart,orderquantity_next_quarter,view_next_quarter,add_to_cart_next_quarter,best_seller_of_quarter,best_seller_next_quarter
0,0,1,2009,42,92750.0,151.35,0,0,39.0,0.0,0.0,0,0.0
1,0,2,2010,39,92750.0,147.46,0,0,14.0,0.0,0.0,0,0.0
3,0,4,2010,14,92750.0,51.56,0,0,13.0,0.0,0.0,0,0.0
4,1,2,2010,17,277250.0,184.09,0,0,40.0,0.0,0.0,0,0.0
6,1,4,2010,40,277250.0,430.88,0,0,23.0,0.0,0.0,0,0.0
7,1,4,2011,23,277250.0,262.31,0,0,25.0,0.0,0.0,0,0.0
8,2,2,2009,82,391750.0,1319.33,0,0,117.0,0.0,0.0,0,1.0
9,2,2,2010,117,391750.0,1791.65,0,0,113.0,0.0,0.0,1,1.0
10,2,2,2011,113,391750.0,1748.35,0,0,74.0,0.0,0.0,1,0.0
11,2,2,2012,74,391750.0,1149.37,0,0,5.0,0.0,0.0,0,0.0


In [93]:
from random import randint

conversion_rate_co = np.random.uniform(0.15, 0.35, size=len(quarter)) 
quarter['add_to_cart'] = (quarter['ORDERQUANTITY'] / conversion_rate_co).round().astype(int)
conversion_rate_vc = np.random.uniform(0.005, 0.03, size=len(quarter)) 
quarter['view'] = (quarter['add_to_cart'] / conversion_rate_vc).round().astype(int)

quarter['add_to_cart'] = quarter['add_to_cart'].clip(lower=0)
quarter['view'] = quarter['view'].clip(lower=0)

quarter.head(10)

,variant_id,quarter,year,ORDERQUANTITY,UNITPRICE,SALES,view,add_to_cart,orderquantity_next_quarter,view_next_quarter,add_to_cart_next_quarter,best_seller_of_quarter,best_seller_next_quarter
0,0,1,2009,42,92750.0,151.35,11791,269,39.0,0.0,0.0,0,0.0
1,0,2,2010,39,92750.0,147.46,6569,178,14.0,0.0,0.0,0,0.0
3,0,4,2010,14,92750.0,51.56,3957,52,13.0,0.0,0.0,0,0.0
4,1,2,2010,17,277250.0,184.09,3474,78,40.0,0.0,0.0,0,0.0
6,1,4,2010,40,277250.0,430.88,9651,207,23.0,0.0,0.0,0,0.0
7,1,4,2011,23,277250.0,262.31,4903,113,25.0,0.0,0.0,0,0.0
8,2,2,2009,82,391750.0,1319.33,16243,434,117.0,0.0,0.0,0,1.0
9,2,2,2010,117,391750.0,1791.65,12890,339,113.0,0.0,0.0,1,1.0
10,2,2,2011,113,391750.0,1748.35,86732,548,74.0,0.0,0.0,1,0.0
11,2,2,2012,74,391750.0,1149.37,16322,213,5.0,0.0,0.0,0,0.0


In [94]:
cols = quarter.columns
cols

Index(['variant_id', 'quarter', 'year', 'ORDERQUANTITY', 'UNITPRICE', 'SALES',
       'view', 'add_to_cart', 'orderquantity_next_quarter',
       'view_next_quarter', 'add_to_cart_next_quarter',
       'best_seller_of_quarter', 'best_seller_next_quarter'],
      dtype='object')

In [95]:
quarter.columns = quarter.columns.str.lower()

In [96]:
quarter.head()

,variant_id,quarter,year,orderquantity,unitprice,sales,view,add_to_cart,orderquantity_next_quarter,view_next_quarter,add_to_cart_next_quarter,best_seller_of_quarter,best_seller_next_quarter
0,0,1,2009,42,92750.0,151.35,11791,269,39.0,0.0,0.0,0,0.0
1,0,2,2010,39,92750.0,147.46,6569,178,14.0,0.0,0.0,0,0.0
3,0,4,2010,14,92750.0,51.56,3957,52,13.0,0.0,0.0,0,0.0
4,1,2,2010,17,277250.0,184.09,3474,78,40.0,0.0,0.0,0,0.0
6,1,4,2010,40,277250.0,430.88,9651,207,23.0,0.0,0.0,0,0.0


In [97]:
# TÍNH TOÁN CÁC CHỈ SỐ TỶ LỆ CHUYỂN ĐỔI (Conversion Ratios)
# ----------------------------------------------------------------------
# 1. Tỷ lệ Thêm vào Giỏ hàng: Số lượng thêm vào giỏ / Lượt xem
quarter['add_to_cart_rate'] = quarter["add_to_cart"] / quarter["view"]

# 2. Tỷ lệ Chuyển đổi từ Giỏ hàng sang Đặt hàng: Số lượng đặt hàng / Số lượng thêm vào giỏ
# Sử dụng ORDERQUANTITY (Số lượng đặt hàng)
quarter["cart_to_order_rate"] = quarter["orderquantity"] / quarter["add_to_cart"]

# 3. Tỷ lệ Chuyển đổi tổng thể: Số lượng đặt hàng / Lượt xem
quarter["conversion_rate"] = quarter["orderquantity"] / quarter["view"]

# 4. Giá trị Đơn hàng Trung bình (AOV - Average Order Value): Tổng Doanh số / Tổng Số lượng đặt hàng
quarter["aov"] = quarter["sales"] / quarter["orderquantity"]


# TÍNH TOÁN CÁC CHỈ SỐ TĂNG TRƯỞNG (Growth Rates)
# ----------------------------------------------------------------------
# LƯU Ý: Thay thế '_next_month' thành '_next_quarter'

# 5. Tăng trưởng Số lượng Đặt hàng (Quý tiếp theo so với Quý hiện tại)
quarter["order_growth"] = quarter["orderquantity_next_quarter"] / quarter["orderquantity"]

# 6. Tăng trưởng Lượt xem (Quý tiếp theo so với Quý hiện tại)
quarter["view_growth"] = quarter["view_next_quarter"] / quarter["view"]

# 7. Tăng trưởng Thêm vào Giỏ hàng (Quý tiếp theo so với Quý hiện tại)
quarter["add_to_cart_growth"] = quarter["add_to_cart_next_quarter"] / quarter["add_to_cart"]

In [98]:
# Đảm bảo tất cả các tính năng được tính trên DataFrame hàng quý 'quarter'
# Đảm bảo sử dụng tên cột chính xác: 'ORDERQUANTITY', 'SALES' (viết hoa)

# -------------------------------------------------------------
# 1. BIẾN TRỄ ĐƠN LẺ (LAG FEATURES)
# -------------------------------------------------------------

# AOV (Giá trị đơn hàng trung bình) trễ 1 quý
quarter["aov_lag_1"] = quarter.groupby("variant_id")["aov"].shift(1)

# Số lượng thêm vào giỏ hàng trễ 1 quý (Tính trực tiếp trên quarter)
quarter["add_to_cart_lag_1"] = quarter.groupby("variant_id")["add_to_cart"].shift(1)

# Số lượng đặt hàng trễ 1, 2, 3 quý
quarter["orderquantity_lag_1"] = quarter.groupby("variant_id")["orderquantity"].shift(1)
quarter["orderquantity_lag_2"] = quarter.groupby("variant_id")["orderquantity"].shift(2)
quarter["orderquantity_lag_3"] = quarter.groupby("variant_id")["orderquantity"].shift(3)

# Lượt xem trễ 1 quý
quarter["view_lag_1"] = quarter.groupby("variant_id")["view"].shift(1)


# -------------------------------------------------------------
# 2. TRUNG BÌNH TRƯỢT (ROLLING AVERAGES)
# -------------------------------------------------------------

# Trung bình Số lượng đặt hàng trong 3 quý gần nhất
# *Đã sửa: sử dụng tên cột 'ORDERQUANTITY'*
quarter["order_3q_avg"] = quarter.groupby("variant_id")["orderquantity"].rolling(3).mean().reset_index(level=0, drop=True)

# Trung bình Số lượng đặt hàng trong 6 quý gần nhất
# *Đã sửa: sử dụng tên cột 'ORDERQUANTITY'*
quarter["order_6q_avg"] = quarter.groupby("variant_id")["orderquantity"].rolling(6).mean().reset_index(level=0, drop=True)

# Trung bình Lượt xem trong 3 quý gần nhất
# *Đã sửa: Đảm bảo tính trên DataFrame 'quarter'*
quarter["view_3q_avg"] = quarter.groupby("variant_id")["view"].rolling(3).mean().reset_index(level=0, drop=True)

# Trung bình Thêm vào Giỏ hàng trong 3 quý gần nhất
quarter["add_to_cart_3q_avg"] = quarter.groupby("variant_id")["add_to_cart"].rolling(3).mean().reset_index(level=0, drop=True)

print("Đã hoàn thành Feature Engineering nâng cao.")

Đã hoàn thành Feature Engineering nâng cao.


In [99]:
quarter.head()

,variant_id,quarter,year,orderquantity,unitprice,sales,view,add_to_cart,orderquantity_next_quarter,view_next_quarter,...,aov_lag_1,add_to_cart_lag_1,orderquantity_lag_1,orderquantity_lag_2,orderquantity_lag_3,view_lag_1,order_3q_avg,order_6q_avg,view_3q_avg,add_to_cart_3q_avg
0,0,1,2009,42,92750.0,151.35,11791,269,39.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2,2010,39,92750.0,147.46,6569,178,14.0,0.0,...,3.603571,269.0,42.0,NaN,NaN,11791.0,NaN,NaN,NaN,NaN
3,0,4,2010,14,92750.0,51.56,3957,52,13.0,0.0,...,3.781026,178.0,39.0,42.0,NaN,6569.0,31.666667,NaN,7439.0,166.333333
4,1,2,2010,17,277250.0,184.09,3474,78,40.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,4,2010,40,277250.0,430.88,9651,207,23.0,0.0,...,10.828824,78.0,17.0,NaN,NaN,3474.0,NaN,NaN,NaN,NaN


In [100]:
quarter.to_csv("dataset_cleaned.csv", index=False)

In [101]:
y = quarter["orderquantity_next_quarter"]

In [102]:
features = [
    # Cấp độ thời gian
    "year",
    "quarter", # Thêm cột 'quarter' nếu bạn muốn nắm bắt tính mùa vụ

    # Giá trị của Quý hiện tại (t)
    "orderquantity",  # Đã sửa
    "unitprice",
    "view",
    "sales",          # Đã sửa (nên là SALES nếu có)
    "add_to_cart",
    "add_to_cart_rate",
    "cart_to_order_rate",
    "conversion_rate",
    "aov",

    # Biến tăng trưởng (Growth features)
    # Các biến này cũng là biến hiện tại (t) được tính từ t+1 / t
    "order_growth",
    "view_growth",
    "add_to_cart_growth",

    # Biến trễ (Lag features) - Quý trước đó (t-1, t-2, t-3)
    "orderquantity_lag_1", # Đây là tên bạn đã sử dụng cho ORDERQUANTITY trễ
    "orderquantity_lag_2",
    "orderquantity_lag_3",
    "view_lag_1",
    "add_to_cart_lag_1",
    "aov_lag_1",


    # Trung bình trượt (Rolling windows) - Đã sửa tên biến cho phù hợp với quý (q)
    "order_3q_avg",
    "order_6q_avg",
    "view_3q_avg",
    "add_to_cart_3q_avg"
]

In [103]:
X = quarter[features]
y = quarter["orderquantity_next_quarter"]


In [104]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=42
)


In [105]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Create model
model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective='reg:squarederror',
    eval_metric='rmse',                 
    early_stopping_rounds=200           
)

# Fit model
model.fit(
    X_train, 
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=True
)


[0]	validation_0-rmse:20.95795	validation_1-rmse:20.54449
[1]	validation_0-rmse:20.16686	validation_1-rmse:19.77854
[2]	validation_0-rmse:19.33608	validation_1-rmse:19.02529
[3]	validation_0-rmse:18.58821	validation_1-rmse:18.34169
[4]	validation_0-rmse:17.83889	validation_1-rmse:17.63143
[5]	validation_0-rmse:17.13096	validation_1-rmse:16.95243
[6]	validation_0-rmse:16.42685	validation_1-rmse:16.28621
[7]	validation_0-rmse:15.77941	validation_1-rmse:15.64915
[8]	validation_0-rmse:15.21009	validation_1-rmse:15.11405
[9]	validation_0-rmse:14.65836	validation_1-rmse:14.61812
[10]	validation_0-rmse:14.10410	validation_1-rmse:14.11048
[11]	validation_0-rmse:14.05294	validation_1-rmse:14.10821
[12]	validation_0-rmse:13.51324	validation_1-rmse:13.59320
[13]	validation_0-rmse:12.98782	validation_1-rmse:13.09811
[14]	validation_0-rmse:12.49023	validation_1-rmse:12.61407
[15]	validation_0-rmse:12.06693	validation_1-rmse:12.20429
[16]	validation_0-rmse:11.65079	validation_1-rmse:11.81739
[17]	va

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,200
,enable_categorical,False
,eval_metric,'rmse'


In [106]:
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    median_absolute_error
)
import numpy as np

# Prediction
pred = model.predict(X_test)

# Evaluation metrics
mae = mean_absolute_error(y_test, pred)
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, pred)
medae = median_absolute_error(y_test, pred)

# MAPE (Mean Absolute Percentage Error)
mape = np.mean(np.abs((y_test - pred) / y_test)) * 100

# SMAPE (Symmetric MAPE)
smape = np.mean(2 * np.abs(y_test - pred) / (np.abs(y_test) + np.abs(pred))) * 100

# Print results
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R² Score:", r2)
print("Median AE:", medae)
print("MAPE (%):", mape)
print("SMAPE (%):", smape)


MAE: 1.2919743631390899
MSE: 9.040646414691274
RMSE: 3.006766770917105
R² Score: 0.9803034156852755
Median AE: 0.5305042266845703
MAPE (%): 5.647174036378894
SMAPE (%): 5.161943965221172


In [107]:
quarter["predicted_order_next_quarter"] = model.predict(X)


In [108]:
quarter[["variant_id","orderquantity","orderquantity_next_quarter", "predicted_order_next_quarter"]].head(n=500)


,variant_id,orderquantity,orderquantity_next_quarter,predicted_order_next_quarter
0,0,42,39.0,39.070789
1,0,39,14.0,13.909854
3,0,14,13.0,13.059976
4,1,17,40.0,40.245964
6,1,40,23.0,23.108610
...,...,...,...,...
632,144,42,22.0,21.787491
634,144,22,49.0,49.039310
633,144,49,26.0,25.679564
637,145,11,27.0,27.303467


In [109]:
import joblib

joblib.dump(model, "model.pkl")


['model.pkl']